#### Security & Environment Setup

In [12]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve credentials
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')

print("Environment variables loaded")

Environment variables loaded


#### Data Preparation

In [3]:
try:
    df = pd.read_csv('cleaned_retail_data.csv')
    print(f"Successfully loaded {len(df)} cleaned records.")
except FileNotFoundError:
    print("Error: 'cleaned_retail_data.csv' not found.")

Successfully loaded 5000 cleaned records.


#### Secure Database Connection

In [5]:
from sqlalchemy import text 

# 1. Create a connection to the MySQL server
connection_url = f"mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}"
engine = create_engine(connection_url)

# 2. Initialize the Database using text() for execution
with engine.connect() as conn:
    # Use the text() function to make the string executable
    conn.execute(text("CREATE DATABASE IF NOT EXISTS Dubai_Retail_DB"))
    # Commit the change to ensure the database is created
    conn.commit() 
    print("Database 'Dubai_Retail_DB' is ready.")

# 3. Re-point engine to the specific database for table creation
db_url = f"mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}/Dubai_Retail_DB"
engine = create_engine(db_url)

Database 'Dubai_Retail_DB' is ready.


#### Data Migration

In [7]:
df.to_sql('stock_performance', con=engine, if_exists='replace', index=False)

print("Migration Successful: The 'stock_performance' table is now live in MySQL.")

Migration Successful: The 'stock_performance' table is now live in MySQL.


#### The Strategic KPI Query 

#### Stock Turnover Analysis

In [8]:
query = """
SELECT 
    Event,
    Product_Name,
    SUM(Sales_Qty) AS Total_Sales,
    AVG(Stock_On_Hand) AS Avg_Inventory,
    -- Calculation: How many times we sold through our average stock
    ROUND(SUM(Sales_Qty) / NULLIF(AVG(Stock_On_Hand), 0), 2) AS Turnover_Rate
FROM stock_performance
GROUP BY Event, Product_Name
ORDER BY Turnover_Rate DESC;
"""

# Execute and load into a results dataframe
kpi_df = pd.read_sql(query, engine)

print("Top 10 High-Velocity Event-Product Combinations:")
kpi_df.head(10)

Top 10 High-Velocity Event-Product Combinations:


,Event,Product_Name,Total_Sales,Avg_Inventory,Turnover_Rate
0,Black Friday,Smart Watch Ultra,21957.00,263.0478,83.47
1,Black Friday,Limited Edition Sneaker,20356.00,270.0000,75.39
2,Black Friday,Organic Saffron Giftset,20029.00,275.1070,72.80
3,Black Friday,Oud Silk Mood Perfume,19582.00,271.8750,72.03
4,Dubai Shopping Festival,Organic Saffron Giftset,17937.00,269.8592,66.47
5,Black Friday,Designer Leather Tote,18321.75,281.3297,65.13
6,Dubai Shopping Festival,Smart Watch Ultra,16207.00,256.0890,63.29
7,Eid Al-Fitr,Organic Saffron Giftset,15860.75,255.1150,62.17
8,Dubai Shopping Festival,Limited Edition Sneaker,16123.00,266.3457,60.53
9,Eid Al-Fitr,Limited Edition Sneaker,15569.00,258.4905,60.23


#### Demographic Purchase Behavior

In [13]:
query_demographics = """
SELECT 
    Store_Location,
    Product_Name,
    COUNT(Transaction_ID) AS Transaction_Count,
    ROUND(AVG(Unit_Price_AED * Sales_Qty), 2) AS Avg_Basket_Value
FROM stock_performance
GROUP BY Store_Location, Product_Name
ORDER BY Avg_Basket_Value DESC;
"""

demo_df = pd.read_sql(text(query_demographics), engine)
print("Demographic Insights by Location:")
demo_df.head()

Demographic Insights by Location:


,Store_Location,Product_Name,Transaction_Count,Avg_Basket_Value
0,Mall of the Emirates,Organic Saffron Giftset,346,80490.65
1,Mall of the Emirates,Limited Edition Sneaker,336,78367.66
2,Dubai Mall,Organic Saffron Giftset,310,77296.34
3,Mall of the Emirates,Smart Watch Ultra,343,76446.02
4,Dubai Marina,Smart Watch Ultra,352,75828.75


#### Identifying "Ghost Inventory" (Data Integrity Check)

In [15]:
# Identifying Stagnant Stock (Irregularities)
query_stagnant = """
SELECT 
    Product_Name,
    Event,
    Stock_On_Hand,
    Sales_Qty
FROM stock_performance
WHERE Sales_Qty = 0 AND Stock_On_Hand > 100
ORDER BY Stock_On_Hand DESC;
"""

stagnant_df = pd.read_sql(text(query_stagnant), engine)
print("Potential Data Irregularities (High Stock / Zero Sales):")
stagnant_df.head()

Potential Data Irregularities (High Stock / Zero Sales):


,Product_Name,Event,Stock_On_Hand,Sales_Qty


#### Business Insights & Recommendations

In [11]:
print("STRATEGIC ANALYSIS FOR STAKEHOLDERS")

# 1. Identify the Top Event
top_event = kpi_df.groupby('Event')['Total_Sales'].sum().idxmax()
print(f"1. HIGHEST REVENUE EVENT: {top_event}. Inventory should be increased by 20% for this period next year.")

# 2. Identify High Turnover Risk
high_risk = kpi_df[kpi_df['Turnover_Rate'] > 75]
print(f"2. STOCK-OUT RISK: {len(high_risk)} products have a Turnover Rate > 75 during Black Friday.")
for product in high_risk['Product_Name'].unique():
    print(f"   - ACTION REQUIRED: Prioritize supply chain for '{product}'.")

# 3. Competitor Partnership Insight
print(f"3. PARTNERSHIP RECOMMENDATION: Target youth-focused brand partnerships for DSF, given the high velocity of Sneakers.")

STRATEGIC ANALYSIS FOR STAKEHOLDERS
1. HIGHEST REVENUE EVENT: Black Friday. Inventory should be increased by 20% for this period next year.
2. STOCK-OUT RISK: 2 products have a Turnover Rate > 75 during Black Friday.
   - ACTION REQUIRED: Prioritize supply chain for 'Smart Watch Ultra'.
   - ACTION REQUIRED: Prioritize supply chain for 'Limited Edition Sneaker'.
3. PARTNERSHIP RECOMMENDATION: Target youth-focused brand partnerships for DSF, given the high velocity of Sneakers.
